# FEMS API Tutorial

This notebook is meant to demonstrate using the FEMS API to retrieve fuel moisture field samples. The code will be deployed through the python module `NAME.py`. The current goal is to read in 100h and 1000h field samples and adapt an RNN trained on 10h sensor data to forecast 100h FEMC via transfer learning. Longer term, this could be used to model live fuel types, duff, and dead moss.

## Setup

In [ ]:
import numpy as np
import pandas as pd

# Local modules
import src.fems_api as fems
from src.utils import read_yml

## Metadata

The module has functions to get the following metadata related to the project. The code queries the API to get all available fields. The intended use of the functions is to query and build metadata files and save to a stash. The stash creation could be rerun on different systems or when the API updates.
- All available sampling locations. Stashed in `data/fems_sts.xslx`
    - Sites can be filtered by spatial bounding box, see `etc/gaccs.yaml` for bounding boxes for NIFC GACCs
- All available fuel types. Stashed in `etc/fuels.yaml`

NOTE: the spatial bounding boxes were used from `wrfxpy` `rtma_cycler`. The actual GACCs are not rectangular, so the bounding boxes buffer the actual areas. Thus, using the bbox values from `etc/gaccs.yaml` will collect some sampling locations that are designated as other GACCs than the one requested.

In [ ]:
sts = fems.get_all_sites(stash_path = "data/fems_sts.xlsx")
fuels = fems.get_fuel_types(stash_path = "etc/fuels.yaml")

In [ ]:
sts.head()

In [ ]:
print(fuels["100-Hour"])

In [ ]:
bbox = read_yml("etc/gaccs.yaml", subkey="Rocky Mountain GACC")["bbox"]
bbox

### Get Stations by BBox

In [ ]:
sts = fems.get_sites_in_bbox(bbox = bbox, stash_path = "data/fems_sts.xlsx")
sts

## Get Samples by Time Range

- Define time period and list of stations (e.g. stations inside a bbox)
- Retrieve sample IDs given those parameters
- Retrieve all available 100h, 1000h, and Duff field samples

In [ ]:
stids = sts.siteId.to_list()
fm_params = {
    "startDate": "2024-06-01T00:00:00+00",
    "endDate": "2024-06-30T23:00:00+00",
    "siteIds": [1352, 1348],
    "fuelTypes": ["100-Hour", "1000-Hour", "Duff"],
}

In [ ]:
results = fems.get_fuel_data(
        fm_params, verbose=True
    )

In [ ]:
results